In [1]:
def get_points(json_info):
    points = []

    if isinstance(json_info, dict):
        for key, value in json_info.items():
            if key == 'points':
                points += value
            else:
                points += get_points(value)

    elif isinstance(json_info, list):
        for element in json_info:
            points += get_points(element)

    return points


def set_points(json_info, points):
    if isinstance(json_info, dict):
        for key, value in json_info.items():
            if key == 'points':
                for i in range(len(value)):
                    value[i] = points.pop(0)
            else:
                set_points(value, points)

    elif isinstance(json_info, list):
        for element in json_info:
            set_points(element, points)


def resize(image, min_size, max_size, resize_if_smaller=False):
    h, w = image.shape[:2]
    min_ratio = min(h, w) / min_size
    max_ratio = max(h, w) / max_size
    ratio = max(min_ratio, max_ratio)
    if resize_if_smaller or ratio > 1:
        image = cv2.resize(image, (0, 0), fx=1 / ratio, fy=1 / ratio)

    return image

In [2]:
def draw_label(image, json_info):
    color = (0, 255, 0)
    thickness = max(image.shape) // 400

    for shape in json_info['shapes']:
        shape_type = shape['shape_type']
        points = np.int32(shape['points'])
        if shape_type == 'polygon':
            cv2.polylines(
                img=image,
                pts=[points],
                isClosed=True,
                color=color,
                thickness=thickness,
            )
        elif shape_type == 'rectangle':
            cv2.rectangle(
                img=image,
                pt1=tuple(points[:2]),
                pt2=tuple(points[2:]),
                color=color,
                thickness=thickness
            )
        elif shape_type == 'line':
            cv2.line(
                img=image,
                pt1=tuple(points[:2]),
                pt2=tuple(points[2:]),
                color=color,
                thickness=thickness
            )

    return image

In [85]:
import cv2
import json
import random
import numpy as np
import imgaug.augmenters as iaa

from pathlib import Path
from imgaug.augmentables import Keypoint
from imgaug.augmentables import KeypointsOnImage
from imgaug.augmentables.segmaps import SegmentationMapsOnImage

In [86]:
bg_images = list(Path('./sample/back_ground/').glob('**/*'))
json_paths = list(Path('./sample/fore_ground/COMMUNIST_PARTY/CARD_BACK/').glob('**/*.json'))

In [95]:
transforms = [
    iaa.Affine(rotate=(-20, 20), shear=(-15, 0), fit_output=True),
#     iaa.Rot90(k=2, keep_size=False),
#     iaa.TranslateX(percent=0.1),
#     iaa.Flipud(p=1),
#     iaa.PiecewiseAffine(scale=(0.05, 0.01)),
    iaa.PerspectiveTransform(scale=(0.01, 0.05), keep_size=False, fit_output=True),
    iaa.ElasticTransformation(alpha=(0, 1.0), sigma=0.25)
]

fg_ratio = random.uniform(0.3, 0.7)

In [96]:
# Choose json file and background image randomly
bg_image = cv2.imread(str(bg_images[0]))
json_path = json_paths[1]

In [100]:
with json_path.open(mode='r', encoding='utf-8') as f:
    json_info = json.load(f)
    
print(type(json_info))

fg_path = str(json_path.with_name(Path(json_info['imagePath']).name))
fg_image = cv2.imread(fg_path)

# Get Force Ground Points from Json Info
points = get_points(json_info)
keypoints = [Keypoint(x=point[0], y=point[1]) for point in points]
keypoints = KeypointsOnImage(keypoints=keypoints, shape=fg_image.shape)

# Get Force Ground Mask
fg_mask = np.ones_like(fg_image)
fg_mask = SegmentationMapsOnImage(fg_mask, fg_image.shape)

print(type(keypoints))
print(type(fg_mask))

# Augment Force Ground Image, Mask, Points
for trans in transforms:
    aug = trans.to_deterministic()
    fg_image = aug(image=fg_image)
    keypoints = aug(keypoints=keypoints)
    fg_mask = aug(segmentation_maps=fg_mask)
    if isinstance(trans, iaa.PerspectiveTransform):
        trans.keep_size=True
        trans.fit_output=False
        bg_image = aug(image=bg_image)

# Set ratio between Force Ground and Back Ground
fg_height, fg_width = fg_image.shape[:2]
bg_height, bg_width = bg_image.shape[:2]

if max(fg_height, fg_width) > min(bg_height, bg_width) * fg_ratio:
    if fg_height >= fg_width:
        fg_height_new = min(bg_height, bg_width) * fg_ratio
        fg_width_new =  fg_height_new * fg_width / fg_height
    else:
        fg_width_new = min(bg_height, bg_width) * fg_ratio
        fg_height_new =  fg_width_new * fg_height / fg_width
    # Resize
    resize_to_size = iaa.Resize({"height": int(fg_height_new), "width": int(fg_width_new)})
    fg_image = resize_to_size(image=fg_image)
    keypoints = resize_to_size(keypoints=keypoints)
    fg_mask = resize_to_size(segmentation_maps=fg_mask)
else:
    bg_width_new, bg_height_new = int(fg_width / fg_ratio), int(fg_height / fg_ratio)
    bg_image = iaa.CropToFixedSize(bg_width_new, bg_height_new)(image=bg_image)

# Pad Force Ground Image, Mask, Points To Fixed Size with Back Ground
pad_to_size = iaa.PadToFixedSize(width=bg_image.shape[1], height=bg_image.shape[0])
pad_to_size = pad_to_size.to_deterministic()
fg_image = pad_to_size(image=fg_image)
keypoints = pad_to_size(keypoints=keypoints)
fg_mask = pad_to_size(segmentation_maps=fg_mask)

# Set All Points to Json Info
points = [[float(keypoint.x), float(keypoint.y)] for keypoint in keypoints.keypoints]
set_points(json_info, points)

# Blend Fore Ground Mask
fg_mask = fg_mask.get_arr().astype(np.float32)
ksize = max(max(fg_image.shape) // 400 * 2 + 1, 5)
fg_mask = cv2.GaussianBlur(fg_mask, ksize=(ksize, ksize), sigmaX=ksize)

# Combinate Force Ground and Back Ground with Mask
image = fg_image * fg_mask + bg_image * (1. - fg_mask)
image = image.astype(np.uint8)

image = iaa.ChangeColorTemperature(kelvin=11000)(image=image)

<class 'dict'>
<class 'imgaug.augmentables.kps.KeypointsOnImage'>
<class 'imgaug.augmentables.segmaps.SegmentationMapsOnImage'>


In [98]:
with Path(Path(json_info['imagePath']).stem + '.json').open(mode='w', encoding='utf-8') as f:
    json.dump(json_info, f, indent=4, ensure_ascii=False)

cv2.imwrite(Path(json_info['imagePath']).name, image)

# cv2.imshow('a', image)
# cv2.waitKey()
# cv2.destroyAllWindows()

True